# Configuration

Funflow provides support for configuring a `Flow` via a YAML config file or environment variables. Support for automatically generated CLI flags is also planned but as of the writing of this tutorial has not been implemented.

If a Task you are using contains a `Configurable` field, when you write your `Flow` you will need to specify the source for that configuration via one of three constructors: `Literal`, `ConfigFromEnv`, or `ConfigFromFile`. For example, the `Arg` field of the `DockerTask` supports configurable args. Let's take a look at a few examples.

## Environment Variables

In [131]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE GADTs #-}
-- Note: Using OverloadedStrings with DockerTask since it will automatically
-- make sure that any `Literal` strings we write are of type `Arg`
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE QuasiQuotes #-}
{-# LANGUAGE RankNTypes #-}

import Control.Exception.Safe (StringException(..), try)
import Control.Monad.Catch (handleIf)
import Data.Maybe (fromMaybe, isJust, isNothing)

import qualified Data.Map as Map
import Path (Abs, Dir, File, Path, parseAbsDir, reldir, relfile, toFilePath, (</>))
import System.Directory (getCurrentDirectory)
import System.Environment (lookupEnv, setEnv, unsetEnv)

import qualified Data.CAS.ContentStore as CS
import Funflow
import Funflow.Tasks.Docker
import Funflow.Config (Configurable (Literal, ConfigFromFile, ConfigFromEnv))

Line 5: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:
Line 6: Unused LANGUAGE pragma
Found:
{-# LANGUAGE QuasiQuotes #-}
Why not:

In [43]:
flow1 = dockerFlow $ 
    DockerTaskConfig {
        image="alpine:latest",
        command="echo",
        args=["this is a hard-coded literal value, the next value is:", Arg $ ConfigFromEnv "CONFIGURING_FLOWS"]
    }

Now we just need to set the `CONFIGURING_FLOWS` environment variable and run the task

In [5]:
setEnv "CONFIGURING_FLOWS" "'hello from an environment variable!'"

runFlow flow1 DockerTaskInput {inputBindings = [], argsVals = mempty} :: IO (CS.Item)

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T23:23:37.067491537Z this is a hard-coded literal value, the next value is: hello from an environment variable!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## YAML File
For passing in configurations via a config file, use the `ConfigFromFile` constructor and pass a config file path to `runFlowWithConfig`.

In [24]:
getStoreAndConfig :: IO (Path Abs Dir, Path Abs File)
getStoreAndConfig = do
    cwd <- parseAbsDir =<< getCurrentDirectory
    let storeDirPath = cwd </> [reldir|./.tmp/store|]
        configFilePath = cwd </> [relfile|./flow.yaml|]
    return (storeDirPath, configFilePath)
    
getTempRunConf :: IO RunFlowConfig
getTempRunConf = (\(d, f) -> RunFlowConfig{ configFile = Just f, storePath = d }) <$> getStoreAndConfig

In [7]:
-- Inspect the config file.
lines <$> ((toFilePath . snd <$> getStoreAndConfig) >>= readFile)

["ourMessage: \"Hello from the flow.yaml\"","ourOtherValue: 42"]

In [32]:
goodFileArg = Arg $ ConfigFromFile "ourMessage"

runWithEmptyInput :: [Arg] -> IO CS.Item
runWithEmptyInput confArgs = 
    let flowConf = DockerTaskConfig{ image = "alpine:latest", command = "echo", args = confArgs }
    in getTempRunConf >>= (\runCfg -> runFlowWithConfig runCfg (dockerFlow flowConf) (mempty :: DockerTaskInput) )

In [33]:
runWithEmptyInput [goodFileArg]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T02:34:50.878291557Z Hello from the flow.yaml
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## Mixing `Configurable`s

As noted, a `Configurable` argument may constructed one of three ways:
* `Literal`
* `ConfigFromEnv`
* `ConfigFromFile`

Since each yields a `Configurable a` value, though, we may mix these when we provide the `args` list to a task configuration.

In [34]:
-- Mixing literal and file config
runWithEmptyInput ["GREETING 1", goodFileArg]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T02:35:05.930536958Z GREETING 1 Hello from the flow.yaml
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

In [35]:
setEnv "SECOND_GREETING" "checkItOut!"
lookupEnv "SECOND_GREETING"

Just "checkItOut!"

In [36]:
-- Mixing config file and env var.
runWithEmptyInput [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING"]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T02:35:12.282644515Z Hello from the flow.yaml checkItOut!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

We can also mix `Placeholder`s with `Configurable`s:

In [46]:
currArgs  = [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING", Placeholder "par3"]
currInput = DockerTaskInput{ inputBindings = [], argsVals = Map.fromList [("par3", "hello-from-arg3")] }
flowConf  = DockerTaskConfig{ image = "alpine:latest", command = "echo", args = currArgs }
getTempRunConf >>= (\runCfg -> runFlowWithConfig runCfg (dockerFlow flowConf) currInput :: IO CS.Item)

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T02:39:33.103011486Z Hello from the flow.yaml checkItOut! hello-from-arg3
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## _Safety for efficiency_: fail fast to avoid lost compute time and resources

Another key point is that _configuration precedes execution_. This means that we can _avoid eventual failure_ by detecting the it during interpretation, before we ever actually run anything. Here we set up simple examples to demonstrate, but imagine the time (and perhaps monetary!) cost that could be saved when failure would occur after a long-running computation.

### Argument existence as a prerequisite

First, note that regardless of how an argument is constructed, execution requires resolution of a value for the argument (though for a `Literal` this is trivial.) To see this, we'll catch exceptions of type `StringException` since that's what should arise when a configuration is illegal or incomplete. Any other result will yield an alarming message. We thus define a helper:

In [73]:
-- First, define a helper for running a simple echo command
import Control.Exception (SomeException)

runEcho1 :: Arg -> Maybe RunFlowConfig -> Maybe DockerTaskInput -> IO ()
runEcho1 arg runConfOpt taskInOpt = 
    let flowConf  = DockerTaskConfig{ image = "alpine:latest", command = "echo", args = [arg] }
        taskIn    = fromMaybe mempty taskInOpt
        getStore  = (</> [reldir|./.tmp/store|]) <$> (getCurrentDirectory >>= parseAbsDir)
        mkRunConf = case runConfOpt of 
            Nothing -> (\s -> RunFlowConfig{ configFile = Nothing, storePath = s }) <$> getStore
            Just rc -> pure rc
    in do
        runCfg <- mkRunConf
        result <- try $ (runFlowWithConfig runCfg (dockerFlow flowConf) taskIn :: IO CS.Item)
        case result of
            Left (StringException msg _) -> putStrLn ("Successfully caught exception: " ++ msg)
            Left ex                      -> putStrLn ("Unexpected error: " ++ show ex)
            Right _                      -> putStrLn "Unintended success :/"

Note that this works when the argument is a literal, and we're simply echoed whatever we provide, and we'll get an "unintended success" message:

In [78]:
runEcho1 "salut, funflow" Nothing Nothing

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T04:59:14.601873180Z salut, funflow
Unintended success :/

But if we set up the arguments as an environment variable, a file-configurable, or a placeholder, we will get an error:

In [87]:
-- Set up env var config failure.
isNothing <$> lookupEnv "MY_TMP_EV"

True

In [88]:
-- failure from environment variable
runEcho1 (Arg $ ConfigFromEnv "MY_TMP_EV") Nothing Nothing

Successfully caught exception: Missing the following required config keys: ["MY_TMP_EV"]

In [86]:
-- failure from file configurable
do
    (d, f) <- getStoreAndConfig
    readFile (toFilePath f) >>= print . lines
    runEcho1 (Arg $ ConfigFromFile "MY_TMP_EV") (Just RunFlowConfig{ configFile = Just f, storePath = d }) Nothing

["ourMessage: \"Hello from the flow.yaml\"","ourOtherValue: 42"]
Successfully caught exception: Missing the following required config keys: ["MY_TMP_EV"]

In [89]:
-- failure from placeholder
runEcho1 (Placeholder "WONT_BE_FILLED") Nothing (Just DockerTaskInput{ inputBindings = [], argsVals = mempty })

Found docker images, pulling...
Pulling docker image: alpine:latest
Successfully caught exception: Docker task failed with configuration errors: ["Unfilled labelWONT_BE_FILLED"]

In [90]:
-- now the positive case, to show that the error catch is specific to an actual error and doesn't just occur generally
runEcho1 (Placeholder "WONT_BE_FILLED") Nothing (Just DockerTaskInput{ inputBindings = [], argsVals = Map.fromList [("WONT_BE_FILLED", "surprise!")] })

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T05:20:05.758836940Z surprise!
Unintended success :/

### _No partial execution_: argument existence prerequisite is total.

In [91]:
-- Set stage for config-time error.
unsetEnv "SECOND_GREETING"
isNothing <$> lookupEnv "SECOND_GREETING"

True

In [92]:
-- Trigger a config-time error.
do
    res <- try (runWithEmptyInput [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING"])
    case res of 
        Left (StringException msg _) -> putStrLn ("Caught error: " ++ msg)
        Right _ -> error "Unexpected success!"

Caught error: Missing the following required config keys: ["SECOND_GREETING"]

Note here that rather execution fails right away (in fact, as the flow is configuring the task, before any execution really begins).
More specifically, rather first `echo`ing the value that our config file assigns to `"ourMessage"`, the flow catches the error before any action occurs (i.e., when the underlying task is configured, rather than as it's running).

## Configuration is dynamic and prompted by a run attempt, not static and fixed by construction.

Also important is that _configuration is dynamic_. Although task execution is decoupled from configuration, a new run triggers a fresh interpretation, which
allows a flow's configuration to be reconsidered before it's run again.

In [122]:
-- pretest
unsetEnv "NOT_STATIC"
isNothing <$> lookupEnv "NOT_STATIC"

True

In [123]:
type DockFlow = Flow DockerTaskInput CS.Item

-- First, a little helper to run flows for this demo
runDynaDemo :: DockFlow -> IO ()
runDynaDemo currFlow = do
    (s, _) <- getStoreAndConfig
    result <- try ( runFlowWithConfig RunFlowConfig{ configFile = Nothing, storePath = s } currFlow (mempty :: DockerTaskInput) :: IO CS.Item )
    case result of
        Left (StringException msg _) -> putStrLn ("flow failed: " ++ msg)
        Right _                      -> putStrLn "flow succeeded!"

In [124]:
-- Build the flow that will first fail, then succeed.
dynaFlow = dockerFlow DockerTaskConfig{ image = "alpine:latest", command = "echo", args = [Arg $ ConfigFromEnv "NOT_STATIC"] }

In [125]:
runDynaDemo dynaFlow

flow failed: Missing the following required config keys: ["NOT_STATIC"]

In [126]:
setEnv "NOT_STATIC" "works now!"
isJust <$> lookupEnv "NOT_STATIC"

True

In [127]:
runDynaDemo dynaFlow

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-28T05:52:00.917013567Z works now!
flow succeeded!